In [1]:
pip install openai

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
from dotenv import load_dotenv
load_dotenv()
api = os.getenv("AA")

In [3]:
from openai import OpenAI
import os

client = OpenAI(api_key=api)

intro_prompt = """
Explain the following concepts in clear technical English using Markdown:

1. What is software architecture?
2. What is a monolithic architecture?
3. What is a microservices-based architecture?

Structure the answer exactly with these top-level headings:

# Software Architecture
# Monolithic Architecture
# Microservices Architecture

For each heading:
- Provide a concise definition.
- List key characteristics.
- Briefly mention main advantages.
- Briefly mention main disadvantages.

Target audience: expert software architects and senior software engineers.
Keep the explanation focused, technically rigorous, and avoid marketing language.
"""

response = client.chat.completions.create(
    model="gpt-4o",  # or the model you are using in the rest of the notebook
    messages=[
        {
            "role": "system",
            "content": "You are a software architect and professor who explains concepts clearly using Markdown."
        },
        {
            "role": "user",
            "content": intro_prompt
        }
    ],
    temperature=0.2,
)

intro_markdown = response.choices[0].message.content

# Save the Markdown to a file so it can be reused later in the notebook or exported
with open("intro_architecture.md", "w", encoding="utf-8") as f:
    f.write(intro_markdown)

print("=== Introductory Markdown generated and saved to 'intro_architecture.md' ===")
print()
print(intro_markdown)


=== Introductory Markdown generated and saved to 'intro_architecture.md' ===

# Software Architecture

## Definition
Software architecture refers to the high-level structure of a software system, encompassing the set of structures needed to reason about the system, which comprise software elements, relations among them, and properties of both. It serves as a blueprint for both the system and the project developing it, defining the system's components and their interactions.

## Key Characteristics
- **Abstraction**: Provides a high-level view of the system, abstracting away from the details of implementation.
- **Modularity**: Divides the system into distinct components or modules, each with specific responsibilities.
- **Scalability**: Facilitates the system's ability to handle growth in workload or scope.
- **Interoperability**: Ensures components can work together and communicate effectively.
- **Performance**: Addresses the system's ability to meet performance requirements.
- **Sec

In [4]:
import os
from openai import OpenAI

client = OpenAI(api_key=api)
repoExport = "chef_"
# Read main.tf content
with open(repoExport + "cloud_evolucion_menor.tf", "r", encoding="utf-8") as f:
    terraform_code = f.read()

# Read IaC rule catalog
with open("IAC.txt", "r", encoding="utf-8") as f:
    iac_rules = f.read()

# Build the user prompt, injecting theoretical context + IaC rules + Terraform code
user_prompt = f"""
THEORETICAL CONTEXT (Markdown, for expert architects):
{intro_markdown}

IAC RULE CATALOG — prioritize this evidence:
{iac_rules}

TERRAFORM CODE (main.tf):
{terraform_code}

TASK:
1) Decide whether main.tf describes a MICROservices architecture (true/false).
2) Justify your assessment with explicit evidence, citing from:
   - [R#] for rule references from the IAC catalog, and
   - [C#] for specific code fragments in main.tf.
   Cover at least:
   - modularity (modules/reuse),
   - independent deployment of services,
   - communication style (async queues/events/APIs),
   - distributed deployment (networks/subnets, multiple services, orchestrators).
3) Explicitly mention negative signals that point towards a monolith or tightly-coupled design
   (single deployment unit, one service, strong shared state, etc.).
4) Provide a clear verdict and a confidence score in [0..1] with a short explanation.
5) At the very end of the answer, emit a compact JSON mini-report in a single code block,
   with the following structure and no extra commentary:

{{
  "microservices": true/false,
  "confidence": <float between 0 and 1>,
  "signals_for": ["...", "..."],
  "signals_against": ["...", "..."]
}}

Be concise, technical, and always cite [R#] and/or [C#] in each justification bullet.
"""

response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {
            "role": "system",
            "content": (
                "You are an expert software architect in Infrastructure as Code and "
                "cloud-native microservices. You reason rigorously and write for expert architects."
            ),
        },
        {
            "role": "user",
            "content": user_prompt,
        },
    ],
    temperature=0.2,
    max_tokens=1200,
)

text = response.choices[0].message.content.strip()
print(text)

# Save the full analysis (narrative + JSON mini-report at the end)
output_filename = repoExport + "architecture_analysis1.txt"
with open(output_filename, "w", encoding="utf-8") as f:
    f.write(text)

print(f"Analysis saved in '{output_filename}'")


1) **Assessment**: The provided `main.tf` does not describe a microservices architecture.

2) **Justification**:

   - **Modularity**: 
     - There is no evidence of modularity in the `main.tf` file. The absence of Terraform modules or separate directories for services indicates a lack of modular design [R2]. The code does not demonstrate separation by service or function, which is a hallmark of microservices architecture.
   
   - **Independent Deployment of Services**:
     - The Terraform configuration does not specify any independent deployment units such as ECS services, Lambda functions, or Kubernetes deployments [R1]. The configuration appears to be more focused on infrastructure setup rather than defining deployable services.
   
   - **Communication Style**:
     - There is no evidence of asynchronous communication mechanisms such as SQS, SNS, or EventBridge [R4]. The lack of these elements suggests a potential reliance on synchronous communication, which is more typical of m

In [5]:
import zipfile
import os
from openai import OpenAI

# Initialize client
client = OpenAI(api_key=api)  # <- add your key or use env var

# Read the previous Terraform-based analysis (try English name first, then Spanish as fallback)
previous_analysis = ""
try:
    with open(repoExport + "architecture_analysis1.txt", "r", encoding="utf-8", errors="replace") as f:
        previous_analysis = f.read()
except FileNotFoundError:
    with open(repoExport + "analisis_arquitectura1.txt", "r", encoding="utf-8", errors="replace") as f:
        previous_analysis = f.read()

# Read relevant files from the ZIP
zip_path = "chefapp.zip"
source_code_content = ""

with zipfile.ZipFile(zip_path, "r") as zip_ref:
    for file_info in zip_ref.infolist():
        if file_info.filename.endswith(".py") or file_info.filename.endswith(".tf"):
            with zip_ref.open(file_info.filename) as file:
                try:
                    content = file.read().decode("utf-8", errors="replace")
                    source_code_content += f"\n# ===== File: {file_info.filename} =====\n{content}\n"
                except Exception as e:
                    print(f"Error reading {file_info.filename}: {e}")

# Build the prompt for GPT, preserving theoretical context + previous analysis
prompt = f"""
You are given three sources of information about a hybrid solution (monolith + microservices):

1. A theoretical introduction in Markdown about software architecture, monolithic architecture, and microservices architecture.
2. A previous architecture analysis derived from the Terraform main.tf file.
3. The actual source code used in the solution (Python and Terraform files inside a ZIP archive).

Use ALL of them as context.

== THEORETICAL CONTEXT ==
{intro_markdown}

== PREVIOUS TERRAFORM-BASED ANALYSIS ==
{previous_analysis}

== SOURCE CODE (PY / TF) ==
{source_code_content}

Your tasks:

1. Validate or correct the previous Terraform-based analysis using the real source code.
2. Identify additional architectures or patterns present (e.g., Strategy, decoupling, serverless, messaging, CQRS, etc.).
3. Write an improved architecture analysis that:
   - Clearly describes the current architecture.
   - Explains how and why it is hybrid (monolith + microservices), if applicable.
   - Highlights key quality attributes (scalability, maintainability, performance, security, etc.).
   - Discusses its potential evolution towards a more microservices-based or serverless architecture.

Target audience: expert software architects.
Return the answer in well-structured Markdown, with clear headings and sections.
"""

# Call OpenAI to generate the improved analysis
response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {
            "role": "system",
            "content": "You are a software architect expert in hybrid architectures, Infrastructure as Code, and design patterns."
        },
        {
            "role": "user",
            "content": prompt
        }
    ],
    temperature=0.4,
    max_tokens=2000
)

# Show response
print(response.choices[0].message.content)

# Save improved analysis
improved_answer = response.choices[0].message.content.strip()
output_file = repoExport + "architecture_analysis_improved1.txt"
with open(output_file, "w", encoding="utf-8") as f:
    f.write(improved_answer)

print(f"Improved analysis saved in '{output_file}'")


# Improved Architecture Analysis

## 1. Validation of Previous Terraform-Based Analysis

### Validation of Modularity
The source code confirms the previous analysis regarding the lack of modularity. The Terraform configuration does not utilize modules or separate directories for different services, which is a key feature of microservices architecture. The codebase is structured in a way that suggests a monolithic approach, with a single deployment unit and shared resources.

### Validation of Independent Deployment
The source code does not include configurations for independent deployment units such as ECS services, Lambda functions, or Kubernetes pods. This aligns with the previous analysis that highlighted the absence of independent deployment capabilities, further indicating a monolithic architecture.

### Validation of Communication Style
The source code lacks any implementation of asynchronous communication mechanisms like SQS, SNS, or EventBridge. This supports the previous analy

In [9]:
import os
from openai import OpenAI

client = OpenAI(api_key=api)

path_evol = repoExport + "cloud_evolucion.tf"
path_mayor = repoExport + "cloud_evolucion_mayor.tf"

if os.path.exists(path_evol):
    with open(path_evol, "r", encoding="utf-8") as f:
        terraform_code = f.read()
elif os.path.exists(path_mayor):
    with open(path_mayor, "r", encoding="utf-8") as f:
        terraform_code = f.read()
else:
    raise FileNotFoundError(
        f"No se encontró ni {path_evol} ni {path_mayor}"
    )

# Read IaC rule catalog
with open("IAC.txt", "r", encoding="utf-8") as f:
    iac_rules = f.read()

# Build the user prompt, reusing theoretical context + IaC rules + Terraform code
user_prompt = f"""
THEORETICAL CONTEXT (Markdown, for expert architects):
{intro_markdown}

IAC RULE CATALOG — prioritize this evidence:
{iac_rules}

TERRAFORM CODE (main_microservicios.tf):
{terraform_code}

TASK:
1) Decide whether main_microservicios.tf describes a MICROservices architecture (true/false).
2) Justify your assessment with explicit evidence, citing from:
   - [R#] for rule references from the IaC catalog, and
   - [C#] for specific code fragments in main_microservicios.tf.
   Cover at least:
   - modularity (modules/reuse),
   - independent deployment of services,
   - communication style (async queues/events/APIs),
   - distributed deployment (networks/subnets, multiple services, orchestrators).
3) Explicitly mention negative signals that point towards a monolith or tightly-coupled design
   (single deployment unit, one service, strong shared state, etc.).
4) Provide a clear verdict and a confidence score in [0..1] with a short explanation.
5) At the very end of the answer, emit a compact JSON mini-report in a single code block,
   with the following structure and no extra commentary:

{{
  "microservices": true/false,
  "confidence": <float between 0 and 1>,
  "signals_for": ["...", "..."],
  "signals_against": ["...", "..."]
}}

Be concise, technical, and always cite [R#] and/or [C#] in each justification bullet.
"""

response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {
            "role": "system",
            "content": (
                "You are an expert software architect in Infrastructure as Code and "
                "cloud-native microservices. You reason rigorously and write for expert architects."
            ),
        },
        {
            "role": "user",
            "content": user_prompt,
        },
    ],
    temperature=0.2,
    max_tokens=1200,
)

text = response.choices[0].message.content.strip()
print(text)

# Save the full analysis (narrative + JSON mini-report at the end)
output_filename = repoExport + "architecture_analysis2.txt"
with open(output_filename, "w", encoding="utf-8") as f:
    f.write(text)

print(f"Analysis saved in '{output_filename}'")


1) **Assessment**: The `main_microservicios.tf` describes a microservices architecture.

2) **Justification**:

   - **Modularity**:
     - The Terraform code is structured into multiple modules such as `aws`, `efs`, `s3`, `aws-output`, etc. [C#]. This indicates a modular design where each module encapsulates specific functionalities, aligning with microservices principles [R2].
     - The use of separate modules for different services and infrastructure components suggests a high degree of modularity [R2].

   - **Independent Deployment of Services**:
     - The presence of distinct AWS resources for different services, such as `aws_instance` for `chef_automate`, `chef_server`, `chef_automate_postgresql`, and `chef_automate_opensearch` [C#], supports independent deployment. Each service can be scaled and managed independently [R3].
     - The use of separate ALBs for `automate_lb` and `chef_server_lb` [C#] further supports independent deployment and scaling of services [R3].

   - **C

In [10]:
import zipfile
import os
from openai import OpenAI

# Initialize client
client = OpenAI(api_key=api)

# Read the previous Terraform-based analysis for the microservices architecture
# Try the English filename first, then fall back to the original Spanish name
previous_analysis = ""
try:
    with open(repoExport + "architecture_analysis2.txt", "r", encoding="utf-8", errors="replace") as f:
        previous_analysis = f.read()
except FileNotFoundError:
    with open(repoExport + "analisis_arquitectura2.txt", "r", encoding="utf-8", errors="replace") as f:
        previous_analysis = f.read()

# Read relevant files from the ZIP (microservices_app.zip)
zip_path = "chefapp.zip"
source_code_content = ""

with zipfile.ZipFile(zip_path, "r") as zip_ref:
    for file_info in zip_ref.infolist():
        if file_info.filename.endswith(".py") or file_info.filename.endswith(".tf"):
            with zip_ref.open(file_info.filename) as file:
                try:
                    content = file.read().decode("utf-8", errors="replace")
                    source_code_content += f"\n# ===== File: {file_info.filename} =====\n{content}\n"
                except Exception as e:
                    print(f"Error reading {file_info.filename}: {e}")

# Build the prompt for GPT, reusing theoretical context + previous analysis
prompt = f"""
You are given three sources of information about a microservices-based solution:

1. A theoretical introduction in Markdown about software architecture, monolithic architecture, and microservices architecture.
2. A previous architecture analysis derived from the Terraform file main_microservicios.tf.
3. The actual source code used in the solution (Python and Terraform files inside a ZIP archive).

Use ALL of them as context.

== THEORETICAL CONTEXT ==
{intro_markdown}

== PREVIOUS TERRAFORM-BASED ANALYSIS (MICROSERVICES) ==
{previous_analysis}

== SOURCE CODE (PY / TF) ==
{source_code_content}

Your tasks:

1. Validate or correct the previous Terraform-based analysis using the real source code of the microservices application.
2. Identify additional architectural patterns or styles present (e.g., messaging, event-driven, CQRS, API Gateway patterns, serverless, etc.).
3. Write an improved architecture analysis that:
   - Clearly describes the current microservices architecture.
   - Identifies the main services, their responsibilities, and how they communicate (sync/async, protocols, integration mechanisms).
   - Highlights key quality attributes (scalability, resilience, maintainability, observability, security, etc.).
   - Discusses strengths and weaknesses of the current design and potential improvements.

Target audience: expert software architects.
Return the answer in well-structured Markdown, with clear headings and sections.
"""

# Call OpenAI to generate the improved analysis
response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {
            "role": "system",
            "content": "You are a software architect expert in microservices architectures, Infrastructure as Code, and design patterns."
        },
        {
            "role": "user",
            "content": prompt
        }
    ],
    temperature=0.4,
    max_tokens=2000
)

# Show response
print(response.choices[0].message.content)

# Save improved analysis
improved_answer = response.choices[0].message.content.strip()
output_file = repoExport + "architecture_analysis_improved2.txt"
with open(output_file, "w", encoding="utf-8") as f:
    f.write(improved_answer)

print(f"Improved analysis saved in '{output_file}'")


```markdown
# Improved Architecture Analysis of the Microservices-Based Solution

## 1. Validation of Previous Terraform-Based Analysis

The previous Terraform-based analysis provides a solid foundation for understanding the microservices architecture described in the `main_microservicios.tf` file. Upon reviewing the actual source code, the following validations and corrections are made:

### Validation

- **Modularity**: The source code confirms the modularity described in the Terraform analysis. The use of distinct Terraform modules (`aws`, `efs`, `s3`, etc.) aligns with the principles of microservices, where each module encapsulates specific functionalities.
  
- **Independent Deployment**: The presence of distinct AWS resources (`aws_instance` for `chef_automate`, `chef_server`, etc.) is validated. The source code supports independent deployment and scaling, as each service is managed separately.

- **Distributed Deployment**: The use of multiple subnets and availability zones is c

In [11]:
import os
from openai import OpenAI

# Initialize client
client = OpenAI(api_key=api)  

# Load both improved analyses (HYBRID and MICROSERVICES)
with open(repoExport + "architecture_analysis_improved1.txt", "r", encoding="utf-8", errors="replace") as f:
    analysis_hybrid = f.read()

with open(repoExport + "architecture_analysis_improved2.txt", "r", encoding="utf-8", errors="replace") as f:
    analysis_microservices = f.read()

# Build the prompt for OpenAI (ADR with Motivation / Main Decision / Alternatives / Pros / Cons)
prompt = f"""
You are given two versions of an architecture analysis for the same application,
each one coming from a different implementation and infrastructure:

- File architecture_analysis_improved1.txt describes a hybrid architecture
  with a monolithic component and a single Lambda function.
- File architecture_analysis_improved2.txt describes the same application
  fully migrated to a microservices-based architecture, with multiple Lambda
  functions and an API Gateway.

If available in this session, you may also use the following theoretical introduction
about software architecture, monolithic architecture, and microservices architecture:

== THEORETICAL CONTEXT (OPTIONAL) ==
{intro_markdown if 'intro_markdown' in globals() else '(no theoretical context available in this run)'}

== ARCHITECTURE ANALYSIS - HYBRID VERSION (architecture_analysis_improved1.txt) ==
{analysis_hybrid}

== ARCHITECTURE ANALYSIS - MICROSERVICES VERSION (architecture_analysis_improved2.txt) ==
{analysis_microservices}

Your task is:

1. Identify the most important architecture decisions involved in the migration
   from the HYBRID version to the MICROSERVICES version.
2. For each of these decisions, write an Architecture Decision Record (ADR)
   using the following template, which combines MADR-style elements with the
   expert-recommended sections.

For each ADR, follow these rules:

- Start the ADR with a top-level heading of the form:
  "# ADR: <short decision name>"

- Then, include the following sections in this exact order, with these headings:

## Title
## Status
## Motivation
## Decision Drivers
## Main Decision
## Alternatives
## Pros
## Cons
## Consequences
## Validation
## Additional Information

SECTION DEFINITIONS AND RULES
-----------------------------

- **Title**
  - Short and descriptive of the purpose of the architecture decision.
  - Avoid unnecessary technology names that are not central to the decision.

- **Status**
  - Must be one of: Proposed, Accepted, Rejected, Deprecated, Superseded.
  - Choose the one that best matches the information you can infer.

- **Motivation**
  - Explain the problem being solved by the decision.
  - Blend system context, constraints, and requirements.
  - Clearly describe WHY a decision is needed now.
  - Write continuous prose (no bullet points).

- **Decision Drivers**
  - Bullet list of the main drivers of the decision:
    - functional requirements,
    - non-functional requirements (quality attributes),
    - constraints (organizational, technical, regulatory, etc.).

- **Main Decision**
  - Describe the chosen architecture decision in detail.
  - Explain how it addresses the motivation and decision drivers.
  - Include any relevant assumptions and clarifications.
  - Write continuous prose (no bullet points).

- **Alternatives**
  - List other architecture options that could have addressed the same problem,
    but were not chosen.
  - Do NOT repeat the main decision here.
  - For each alternative, provide a short name and a brief one-line description.

- **Pros**
  - For EACH decision (main decision and alternatives):
    - Create a subheading with the decision/option name and list its advantages.
  - Example structure:
    - Main decision:
      - Pros:
        - ...
        - ...
    - Alternative 1:
      - Pros:
        - ...
        - ...

- **Cons**
  - For EACH decision (main decision and alternatives):
    - Create a subheading with the decision/option name and list its disadvantages.
  - Example structure:
    - Main decision:
      - Cons:
        - ...
        - ...
    - Alternative 1:
      - Cons:
        - ...
        - ...

- **Consequences**
  - Describe positive and negative consequences and trade-offs of the chosen
    main decision, including:
    - short-term vs long-term impact,
    - impact on key quality attributes (scalability, performance, maintainability,
      security, resilience, etc.).

- **Validation**
  - Include this section only if you can reasonably infer how the decision can be
    or has been validated (e.g., tests, prototypes, benchmarks, reviews).
  - If nothing is known, you may write:
    - "Validation to be defined in future iterations."

- **Additional Information**
  - Use this section only for:
    - references,
    - links,
    - related ADRs,
    - issue or pull request IDs,
    - other notes that do not fit in previous sections.
  - If there is nothing relevant, you may leave it empty or omit it.

Additional guidelines:

- Focus on decisions that are clearly implied by the differences between the HYBRID
  and MICROSERVICES versions (for example, migration strategy, decomposition approach,
  communication style, deployment model, data management).
- Do not invent technologies or details that are not supported by the analyses.
- Limit the output to at most 5 ADRs that capture the key decisions in this migration.
- If you detect more than one important decision, produce multiple ADRs, one after another.
- Separate each ADR visually by starting each one with a line that begins with "# ADR:".

Return ONLY the ADRs in Markdown format, with no additional explanation or commentary.
"""

# Send to OpenAI for ADR generation
response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {
            "role": "system",
            "content": "You are a software architect expert in documenting architecture decisions (ADR) using a MADR-inspired template that includes Motivation, Main Decision, Alternatives, Pros, and Cons as separate sections."
        },
        {
            "role": "user",
            "content": prompt
        }
    ],
    temperature=0.3,
    max_tokens=2000
)

# Full text with all ADRs
adr_text = response.choices[0].message.content.strip()
print(adr_text)

# Split ADRs by detecting lines that start with '# ADR'
adr_blocks = []
current_adr_lines = []

for line in adr_text.splitlines():
    if line.strip().lower().startswith("# adr"):
        # When a new ADR starts, close the previous one (if any)
        if current_adr_lines:
            adr_blocks.append("\n".join(current_adr_lines).strip())
            current_adr_lines = []
    current_adr_lines.append(line)

# Add the last ADR block (if any)
if current_adr_lines:
    adr_blocks.append("\n".join(current_adr_lines).strip())

# Save each ADR into a separate file
output_paths = []
for idx, block in enumerate(adr_blocks, start=1):
    filename = repoExport + f"ADR_{idx}.txt"  # or ".md" if you prefer
    with open(filename, "w", encoding="utf-8") as f:
        f.write(block)
    output_paths.append(filename)

print("Generated ADR files:", output_paths)

```markdown
# ADR: Transition to Microservices Architecture

## Title
Transition to Microservices Architecture

## Status
Accepted

## Motivation
The existing hybrid architecture, primarily monolithic with a single Lambda function, limits scalability, flexibility, and resilience. As the application grows, the need for independent scaling, easier maintenance, and improved fault tolerance becomes critical. A microservices architecture promises to address these issues by enabling independent deployment and scaling of services, fostering resilience through distributed deployment, and allowing for more flexible technology adoption.

## Decision Drivers
- Need for improved scalability and flexibility
- Desire for independent deployment and scaling of services
- Requirement for enhanced resilience and fault tolerance
- Organizational shift towards modern architecture practices

## Main Decision
The decision is to migrate from a hybrid monolithic architecture to a fully microservices-based arc

In [12]:
import glob
import json
import os

def parse_adr_markdown(text: str) -> dict:
    """
    Parse a single ADR in Markdown and return a dict with the main fields.
    Assumes the ADR uses:
    # ADR: <name>
    ## Title
    ## Status
    ## Motivation
    ## Decision Drivers
    ## Main Decision
    ## Alternatives
    ## Pros
    ## Cons
    ## Consequences
    ## Validation
    ## Additional Information
    """
    lines = text.splitlines()
    sections = {}
    current_section = None
    adr_name = None

    for line in lines:
        stripped = line.strip()

        # Top-level ADR heading
        if stripped.lower().startswith("# adr"):
            # e.g. "# ADR: Decompose monolith into microservices"
            adr_name = stripped.replace("# ADR:", "").replace("# ADR", "").strip()
            continue

        # Section heading
        if stripped.startswith("## "):
            current_section = stripped[3:].strip()  # remove "## "
            sections[current_section] = []
            continue

        # Accumulate content for current section
        if current_section:
            sections[current_section].append(line)

    def parse_bullet_list(section_name: str):
        """Return all '- ...' or '* ...' lines in the section as a list of strings."""
        raw_lines = sections.get(section_name, [])
        items = []
        for l in raw_lines:
            s = l.strip()
            if s.startswith("-") or s.startswith("*"):
                items.append(s.lstrip("-* ").strip())
        return items

    def join_section(section_name: str):
        """Join all lines of a section into a single cleaned string."""
        raw_lines = sections.get(section_name, [])
        return "\n".join(raw_lines).strip()

    adr_dict = {
        "adr_name": adr_name,
        "title": join_section("Title"),
        "status": join_section("Status"),
        "motivation": join_section("Motivation"),
        "decision_drivers": parse_bullet_list("Decision Drivers"),
        "main_decision": join_section("Main Decision"),
        "alternatives": parse_bullet_list("Alternatives"),
        "pros": join_section("Pros"),
        "cons": join_section("Cons"),
        "consequences": join_section("Consequences"),
        "validation": join_section("Validation"),
        "additional_information": join_section("Additional Information"),
    }

    return adr_dict

# Collect all ADR_*.txt files
adr_files = sorted(glob.glob("serverlessmike_ADR_*.txt"))

adr_json_list = []

for path in adr_files:
    with open(path, "r", encoding="utf-8", errors="replace") as f:
        content = f.read()

    adr_obj = parse_adr_markdown(content)
    adr_obj["source_file"] = os.path.basename(path)
    adr_json_list.append(adr_obj)

# Save all ADRs into a single JSON file
output_json = repoExport + "adr_collection.json"
with open(output_json, "w", encoding="utf-8") as f:
    json.dump(adr_json_list, f, ensure_ascii=False, indent=2)

print(f"Converted {len(adr_json_list)} ADRs to JSON and saved them in '{output_json}'")


Converted 5 ADRs to JSON and saved them in 'chef_adr_collection.json'
